# Penny Stock Model

In [10]:
# import libraries
import pandas as pd
import yfinance as yf
import numpy as np
import datetime

In [11]:
# parameters
VolRise = 0.2  # open position, after current volume will be more than VolRise value
ExitAfter = 0.25 # close position after price achieve ExitAfter
StopLoss = -0.07 # close position after price meets stop
RiskPertrade = 2000 # buy for sum 
TradeAfterDays = 150 # begin trade after TradeAfterDays days from start
price_filter = 0.5 # below this price we open pos



In [12]:
def good_price(ticker_name,date_from_list):
    # function to calculate appropriate price
    res = False
    stock_ticker = yf.Ticker(ticker_name)
    r = stock_ticker.history(max,"1d",date_from_list)
    price = r.iloc[0]["Close"]
    if price<price_filter:
        res = True

    return res


In [13]:
good_price("AEI","2022-01-01")
#reed = yf.Ticker("AEI")
#reed.splits

False

In [14]:
#reed.history("1mo","1d","2022-10-23","2023-01-10")


In [15]:
# load ticker list from excel-file
list_stocks = pd.read_excel('inputFiles/list_stocks.xlsx', index_col ='Company')
all_res = []
data_group=[]
data_group_res=[]

stock_res =[]
stock_small_res=[]
#len(list_stocks)

for i in range(len(list_stocks)): # потім прописати весь список
    symbol = list_stocks.index[i] # беремо акцію по індексу

    datefromlist = list_stocks.iloc[i]['NotDate']
    #print('symbol=',symbol)
    #print('datefromlist=',datefromlist)
    if good_price(symbol,datefromlist)==False:
        continue

    # to calc average volume use 20 days before
    start_date = datefromlist - datetime.timedelta(days=19)                                           
    end_date = datefromlist + datetime.timedelta(days=180)

    

    #load stock price
    #symbol = list_stocks.index[i] # беремо акцію по індексу
    wdf = yf.download(symbol, start_date, end_date)



    # building moving average
    wdf['avgvol']= wdf['Volume'].rolling(window=10, win_type=None).mean()  # середній обєм за 10 днів
    wdf['VolRise'] =wdf['Volume']/wdf['avgvol']-1      # building vol signal
    wdf['volsignal']=wdf['VolRise']>VolRise            # signal column
    wdf['TradeAfter']=wdf.index

    wdf['TradeAfterDaysSig'] = wdf['TradeAfter'] > datefromlist + datetime.timedelta(days=TradeAfterDays)
    wdf['OpenPrice']=0
    wdf['Profit']=0
    wdf['StopPrice']=0
    wdf['TargetPrice']=0
    wdf['Ticker']=symbol
    del(wdf['TradeAfter'])
    lastrow=wdf.tail(1).index # last row in order to stop calculations

    # main module
    PriceOpen = 0
    StopPrice = 0
    TPPrice = 0
    CloseSignal = 0
    status = 0
    
    
    for index, row in wdf.iterrows():
        # looking for opening signal 
        if (PriceOpen ==0) & (lastrow[0]!=index):
            if row['volsignal'] & row['TradeAfterDaysSig']:  # if Volume > than AvgVolume

                PriceOpen = row['Close']  # record close price as price of open
                wdf.at[index,'OpenPrice']= PriceOpen
                StopPrice = PriceOpen-(PriceOpen*StopLoss*-1) # setting stop-loss

                TPPrice = PriceOpen+(PriceOpen*ExitAfter)  # set target price
                status=1

    #     # working with first opened position
        if (PriceOpen >0) & (status==0):
            wdf.at[index,'OpenPrice']= PriceOpen
            wdf.at[index,'StopPrice']= StopPrice
            wdf.at[index,'TargetPrice']= TPPrice


            if row['High']>TPPrice:
                wdf.at[index,'Profit']= RiskPertrade*ExitAfter
                PriceOpen =0
                TPPrice=0
                StopPrice=0

            if row['Low']<StopPrice:
                wdf.at[index,'Profit']= RiskPertrade*StopLoss
                PriceOpen =0
                TPPrice=0
                StopPrice=0


        status = 0    

        # if there is open position and its last row
        if lastrow[0]==index:
            if PriceOpen >0:
                wdf.at[index,'Profit']=(row['Close']/PriceOpen-1)*RiskPertrade
                PriceOpen =0
                TPPrice=0
                StopPrice=0
   
    
    #тут закінчується блок перебору строк акції і йде на наступну акцію
    # 1. тікер
    # 2. profit
    # 3. tradecount
    # 4. ProfitTrades
    #  5.   LossTrades
    # 6.    WinRate
    #7.     AvgHold'
    
    tradecount=(wdf['Profit']!=0).sum()
    ProfitTrades = (wdf['Profit']>0).sum()
    LossTrades = (wdf['Profit']<0).sum()
    #WinRate = round(ProfitTrades/tradecount,1)
    #AvgHold = round((wdf['OpenPrice']!=0).sum()/tradecount,1)
    stock_small_res=[symbol,round(wdf['Profit'].sum(),2),tradecount,ProfitTrades,LossTrades]
    
    data_group=wdf['Profit']
    stock_res.append(stock_small_res)
    data_group_res.append(data_group)
    # блок запису dataFrame:
    all_res.append(wdf)
      
    df_output = pd.concat(all_res)
    
    df_data =pd.concat(data_group_res)
    #df_data.columns['Profit']
   # mask1 = df_data.Profit >0
    #mask2 = df_data.Profit <0
    #df_out= df_data.loc[df_data['Profit']!=0]



    
      
        

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [16]:
df_output['Profit'].sum()

919.8963500442098

In [17]:
stock_df = pd.DataFrame(stock_res, columns = ['Stock','Profit','TradeCount','ProfitTrades','LossTrades'])
df_day = pd.DataFrame(df_data )
df_day.index.name ="Date"

In [18]:
df_day = df_day.loc[df_day['Profit']!=0] # remove 0 values
df_day = df_day.groupby([df_day.index.date]).sum() # групуємо по даті


In [19]:
df_day.sort_index()


,Profit
2022-08-09,10.986823
2022-08-17,-140.000000
2022-08-24,-140.000000
2022-08-26,360.000000
2022-09-01,500.000000
2022-10-21,500.000000
2022-10-26,-140.000000
2022-11-04,-140.000000
2022-11-09,-140.000000
2022-11-11,-140.000000


In [20]:
with pd.ExcelWriter("outputFiles/output.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    df_output.to_excel(writer, sheet_name="full_B")
    df_day.to_excel(writer, sheet_name="by date")
    stock_df.to_excel(writer, sheet_name="statByStock")

In [21]:
#testDF = yf.download("AEI", "2022-10-01", "2022-12-10",back_adjust=True)